# 第8章 分类数据

2020.06.23 1st,  2020.06.27 2nd,  

In [1]:
import pandas as pd
import numpy as np
print(pd.__version__)
df = pd.read_csv('data/table.csv')
df.head()

1.0.5


,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


## 一、category的创建及其性质

### 1. 分类变量的创建

### （a）用Series创建
通过指定 dtype="category" 来直接创建分类变量类型的pandas对象.

In [3]:
pd.Series(["a", "b", "c", "a"], dtype="category")
# Pandas所支持的数据类型: 
# 1. float 
# 2. int 
# 3. bool 
# 4. datetime64[ns] 
# 5. datetime64[ns, tz] 
# 6. timedelta[ns] 
# 7. category 
# 8. object --当一列中有上述多种类型(以及其他Python支持的数据类型)时,会用该类型来进行兼容.
# 默认的数据类型是int64,float64
# 创建panda对象时, 可以指定dtype为上述数据类型的任意一种(但要注意兼容性,例如,不能把含有文本的列指定为数值类型). 如不指定,则会自动推断.

0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): [a, b, c]

In [6]:
pd.Series([10, 0.2545,('A','B'),"a", "b", "c", "a"], dtype="category")
# category类型可以兼容看起来是pandas支持的任意类型的数据类型--这一点有些像 object, 但实现原理则完全不同. 
# 例如,可以将 时间, 时间区间, 字符串, 整型,浮点数,以及浮点数区间等等特殊类型指定为category类型.
# tuple也可以, 但　list 和 dict 不能转为 category 类型 # TypeError: unhashable type: 'list'/dict

0        10
1    0.2545
2    (A, B)
3         a
4         b
5         c
6         a
dtype: category
Categories (6, object): [10, 0.2545, (A, B), a, b, c]

In [11]:
# pd.Series(["a", "b", "c", "a"], dtype="int") 
#ValueError: invalid literal for int() with base 10: 'a'

In [4]:
pd.Series??
#    dtype : str, numpy.dtype, or ExtensionDtype, optional
#        Data type for the output Series. If not specified, this will be
#        inferred from `data`.
#        See the :ref:`user guide <basics.dtypes>` for more usages.

Init signature:
pd.Series(
    data=None,
    index=None,
    dtype=None,
    name=None,
    copy=False,
    fastpath=False,
)
Source:        
class Series(base.IndexOpsMixin, generic.NDFrame):
    """
    One-dimensional ndarray with axis labels (including time series).

    Labels need not be unique but must be a hashable type. The object
    supports both integer- and label-based indexing and provides a host of
    methods for performing operations involving the index. Statistical
    methods from ndarray have been overridden to automatically exclude
    missing data (currently represented as NaN).

    Operations between Series (+, -, /, *, **) align values based on their
    associated index values-- they need not be the same length. The result
    index will be the sorted union of the two indexes.

    Parameters
    ----------
    data : array-like, Iterable, dict, or scalar value
        Contains data stored in Series.

        .. versionchanged:: 0.23.0
           If data is a

In [6]:
np.dtype??

Init signature: np.dtype(self, /, *args, **kwargs)
Docstring:     
dtype(obj, align=False, copy=False)

Create a data type object.

A numpy array is homogeneous, and contains elements described by a
dtype object. A dtype object can be constructed from different
combinations of fundamental numeric types.

Parameters
----------
obj
    Object to be converted to a data type object.
align : bool, optional
    Add padding to the fields to match what a C compiler would output
    for a similar C-struct. Can be ``True`` only if `obj` is a dictionary
    or a comma-separated string. If a struct dtype is being created,
    this also sets a sticky alignment flag ``isalignedstruct``.
copy : bool, optional
    Make a new copy of the data-type object. If ``False``, the result
    may just be a reference to a built-in data-type object.

See also
--------
result_type

Examples
--------
Using array-scalar type:

>>> np.dtype(np.int16)
dtype('int16')

Structured type, one field name 'f1', containing in

In [7]:
pd.DataFrame??
#    dtype : dtype, default None
#        Data type to force. Only a single dtype is allowed. If None, infer.

Init signature:
pd.DataFrame(
    data=None,
    index: Union[Collection, NoneType] = None,
    columns: Union[Collection, NoneType] = None,
    dtype: Union[str, numpy.dtype, ForwardRef('ExtensionDtype'), NoneType] = None,
    copy: bool = False,
)
Source:        
class DataFrame(NDFrame):
    """
    Two-dimensional, size-mutable, potentially heterogeneous tabular data.

    Data structure also contains labeled axes (rows and columns).
    Arithmetic operations align on both row and column labels. Can be
    thought of as a dict-like container for Series objects. The primary
    pandas data structure.

    Parameters
    ----------
    data : ndarray (structured or homogeneous), Iterable, dict, or DataFrame
        Dict can contain Series, arrays, constants, or list-like objects.

        .. versionchanged:: 0.23.0
           If data is a dict, column order follows insertion-order for
           Python 3.6 and later.

        .. versionchanged:: 0.25.0
           If data is a list of

#### （b）对DataFrame指定类型创建

In [7]:
# 将字符串指定为category类型.
temp_df = pd.DataFrame({'A':pd.Series(["a", "b", "c", "d"], dtype="category"),'B':list('abcd')})
temp_df.dtypes

A    category
B      object
dtype: object

In [8]:
temp_df
# 看起来并没有什么区别

,A,B
0,a,a
1,b,b
2,c,c
3,d,d


In [9]:
# 看一下两列的内存使用情况
temp_df.A.memory_usage()

324

In [11]:
temp_df.B.memory_usage()
#不是应该category类型的更省内存吗? ---这是由于所有行都是不一样的,进行 encoding 就会反而占用更多的内存--如果是该列有数万行,但可能的取值只有十种情况, 则分类变量类型会更省内存.

160

In [12]:
# 整数的情况
arr=np.random.randint(1000,1100,100000)
dfa=pd.DataFrame()
dfa['int']=pd.Series(arr,dtype='int')
dfa['str']=pd.Series(arr,dtype='str')
dfa['string']=pd.Series(arr,dtype='str').astype('string')
dfa['O']=pd.Series(arr,dtype='O')
dfa['category']=pd.Series(arr,dtype='category')
dfa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   int       100000 non-null  int32   
 1   str       100000 non-null  object  
 2   string    100000 non-null  string  
 3   O         100000 non-null  object  
 4   category  100000 non-null  category
dtypes: category(1), int32(1), object(2), string(1)
memory usage: 2.8+ MB


In [13]:
dfa['int'].memory_usage(), dfa['str'].memory_usage(), dfa['string'].memory_usage(), dfa['O'].memory_usage(), dfa['category'].memory_usage()
# 这种时候,catagory类型的内存使用量就是最小的了.

(400128, 800128, 800128, 800128, 106048)

In [41]:
# 再举一个长字符串的例子
xy=['孝陵卫皇家理工大学','成都七中附属大学','虹梅南路女子高等师范专科学院','常乐村男子职业技术学校','五道口男子职业技术学院','常乐村男子职业技术学校','翔安皇家理工大学（翔安校区）','中关村文理学院','老和山职业技术学校','沙坡村职业技术学院']
df_xy=pd.DataFrame()
xy_arr=np.random.choice(a=xy,size=100000)# 使用np.random.choice 构造长度10000的Series, 值域为 xy 里的10个值.
# 分别构造各种数据类型的列
df_xy['xy_o']=pd.Series(xy_arr,dtype='O') 
df_xy['xy_str']=pd.Series(xy_arr,dtype='str').astype('str') # 即便加了 astype('str'), 该列的类型还是 object 
df_xy['xy_string']=pd.Series(xy_arr,dtype='string')
df_xy['xy_category']=pd.Series(xy_arr,dtype='category')
# 查看各列的内存使用情况.
df_xy.memory_usage()
# 可以看到, category 类型确实更加节省内存--他的实现方式是对该列的(十个)唯一值进行编码,然后保存一份编码和真实学院的对应关系,再在每个位置保存学院对应的编码.

Index             128
xy_o           800000
xy_str         800000
xy_string      800000
xy_category    100392
dtype: int64

In [30]:
#使用cat.code查看,确实是保存的编码
df_xy['xy_category'].cat.codes
# 可以看到，每一个值都被赋值为一个整数，而且这一列在底层是int8类型
# https://www.jb51.net/article/160670.htm

0        3
1        7
2        0
3        3
4        8
        ..
99995    5
99996    0
99997    3
99998    6
99999    7
Length: 100000, dtype: int8

In [31]:
# 如果数据行数很少,少到和该列的唯一值个数相等, 则使用分类变量时,由于要额外保存一份文本和编码的对照表,而会占用更多的内存
df_xy.drop_duplicates().memory_usage()
# 多出来的那些内存, 就是用来存放对照表的
# 注意到这种情况下, 其他数据类型的内存占用竟然和默认整数的索引一致了 -- 这里一定是存在什么窍门

Index           72
xy_o            72
xy_str          72
xy_string       72
xy_category    401
dtype: int64

In [32]:
df_xy.iloc[:,3:].drop_duplicates().memory_usage()

Index           72
xy_category    401
dtype: int64

In [33]:
# 查看其他的 .cat 类方法
print(dir(df_xy['xy_category'].cat))

['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_accessors', '_add_delegate_accessors', '_constructor', '_delegate_method', '_delegate_property_get', '_delegate_property_set', '_deprecations', '_dir_additions', '_dir_deletions', '_freeze', '_reset_cache', '_validate', 'add_categories', 'as_ordered', 'as_unordered', 'categories', 'codes', 'ordered', 'remove_categories', 'remove_unused_categories', 'rename_categories', 'reorder_categories', 'set_categories']


In [34]:
# 查看编码和具体学院的对应关系--并不是按照每个学院出现顺序进行编码
df_xy['xy_category'].cat.categories

Index(['中关村文理学院', '五道口男子职业技术学院', '孝陵卫皇家理工大学', '常乐村男子职业技术学校', '成都七中附属大学',
       '沙坡村职业技术学院', '翔安皇家理工大学（翔安校区）', '老和山职业技术学校', '虹梅南路女子高等师范专科学院'],
      dtype='object')

In [35]:
df_xy.head()

,xy_o,xy_str,xy_string,xy_category
0,常乐村男子职业技术学校,常乐村男子职业技术学校,常乐村男子职业技术学校,常乐村男子职业技术学校
1,老和山职业技术学校,老和山职业技术学校,老和山职业技术学校,老和山职业技术学校
2,中关村文理学院,中关村文理学院,中关村文理学院,中关村文理学院
3,常乐村男子职业技术学校,常乐村男子职业技术学校,常乐村男子职业技术学校,常乐村男子职业技术学校
4,虹梅南路女子高等师范专科学院,虹梅南路女子高等师范专科学院,虹梅南路女子高等师范专科学院,虹梅南路女子高等师范专科学院


In [36]:
# 使用不同列进行 groupby 操作的耗时
df_xy['cat_codes']= df_xy.xy_category.cat.codes

%timeit df_xy.groupby(df_xy.xy_o)
%timeit df_xy.groupby(df_xy.xy_str)
%timeit df_xy.groupby(df_xy.xy_string)
%timeit df_xy.groupby(df_xy.xy_category) # 为什么用分类数据来分组反而更慢了呢?

%timeit df_xy.groupby(df_xy.xy_category.cat.codes) # 特别是用分类数据的代码分组最慢.

%timeit df_xy.groupby(df_xy.cat_codes) # 先将 cat.codes 作为列放到df_xy中,再按该列分组,就比较快了.


38.6 µs ± 3.77 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
40.3 µs ± 2.71 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
43.1 µs ± 3.42 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
113 µs ± 4.56 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
766 µs ± 32.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
37.9 µs ± 1.98 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [37]:
%timeit df_xy.xy_category.cat.codes # 这一步就比直接按xy_o,xy_str,xy_string分组更费时了.

66.2 µs ± 1.51 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [42]:
df_xy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   xy_o         100000 non-null  object  
 1   xy_str       100000 non-null  object  
 2   xy_string    100000 non-null  string  
 3   xy_category  100000 non-null  category
dtypes: category(1), object(2), string(1)
memory usage: 2.4+ MB


In [45]:
df_xy['xy_str']=df_xy['xy_str'].astype('string').astype('str') # 即便加了 astype('str'), 该列的类型还是 object 
df_xy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   xy_o         100000 non-null  object  
 1   xy_str       100000 non-null  object  
 2   xy_string    100000 non-null  string  
 3   xy_category  100000 non-null  category
dtypes: category(1), object(2), string(1)
memory usage: 2.4+ MB


In [56]:
type0=type(list(df_xy['xy_str'].values)[0])
print(type0)
for i in list(df_xy['xy_str'].values):
    if type(i)==type0:
        pass
    else :
        print(type(i))

<class 'str'>


In [57]:
df_xy['xy_str']=list(df_xy['xy_str'].values) # 即便加了 astype('str'), 该列的类型还是 object 
df_xy.info() # 是因为有括号的原因吗?

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   xy_o         100000 non-null  object  
 1   xy_str       100000 non-null  object  
 2   xy_string    100000 non-null  string  
 3   xy_category  100000 non-null  category
dtypes: category(1), object(2), string(1)
memory usage: 2.4+ MB


#### （c）利用内置Categorical类型创建

In [58]:
cat = pd.Categorical(["a", "b", "c", "a"], categories=['a','b','c'])
pd.Series(cat)

0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): [a, b, c]

In [59]:
type(cat)

pandas.core.arrays.categorical.Categorical

In [38]:
pd.Categorical??
#    Represent a categorical variable in classic R / S-plus fashion.

Init signature:
pd.Categorical(
    values,
    categories=None,
    ordered=None,
    dtype=None,
    fastpath=False,
)
Source:        
class Categorical(ExtensionArray, PandasObject):
    """
    Represent a categorical variable in classic R / S-plus fashion.

    `Categoricals` can only take on only a limited, and usually fixed, number
    of possible values (`categories`). In contrast to statistical categorical
    variables, a `Categorical` might have an order, but numerical operations
    (additions, divisions, ...) are not possible.

    All values of the `Categorical` are either in `categories` or `np.nan`.
    Assigning values outside of `categories` will raise a `ValueError`. Order
    is defined by the order of the `categories`, not lexical order of the
    values.

    Parameters
    ----------
    values : list-like
        The values of the categorical. If categories are given, values not in
        categories will be replaced with NaN.
    categories : Index-like (unique

In [60]:
print(dir(cat))
# 比 Series 对象的方法和模块要少很多.

['T', '__annotations__', '__array__', '__array_priority__', '__array_ufunc__', '__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_accessors', '_can_hold_na', '_concat_same_type', '_constructor', '_deprecations', '_dir_additions', '_dir_deletions', '_dtype', '_formatter', '_from_factorized', '_from_inferred_categories', '_from_sequence', '_from_sequence_of_strings', '_get_codes', '_get_repr', '_internal_get_values', '_maybe_coerce_indexer', '_ndarray_values', '_reduce', '_repr_categories', '_repr_categories_info', '_repr_footer', '_reset_cache', '_reverse_indexer', '_set_categories', '_set_codes', '_set_dtype', '_

In [64]:
print(dir(pd.Series(cat)))

['T', '_AXIS_ALIASES', '_AXIS_IALIASES', '_AXIS_LEN', '_AXIS_NAMES', '_AXIS_NUMBERS', '_AXIS_ORDERS', '_AXIS_REVERSED', '_HANDLED_TYPES', '__abs__', '__add__', '__and__', '__annotations__', '__array__', '__array_priority__', '__array_ufunc__', '__array_wrap__', '__bool__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__div__', '__divmod__', '__doc__', '__eq__', '__finalize__', '__float__', '__floordiv__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__iand__', '__ifloordiv__', '__imod__', '__imul__', '__init__', '__init_subclass__', '__int__', '__invert__', '__ior__', '__ipow__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__long__', '__lt__', '__matmul__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__nonzero__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdiv__', '__rdiv

In [65]:
type(pd.Series(cat))

pandas.core.series.Series

#### （d）利用cut函数创建

#### 默认使用区间类型为标签

In [67]:
pd.cut(np.random.randint(0,60,5), bins=[0,10,30,60])

[(30, 60], (10, 30], (30, 60], (10, 30], (10, 30]]
Categories (3, interval[int64]): [(0, 10] < (10, 30] < (30, 60]]

In [71]:
type(pd.cut(np.random.randint(0,60,5), bins=[0,10,30,60]))

pandas.core.arrays.categorical.Categorical

In [72]:
# 转为 Series, 该列的数据类型还是分类变量.
pd.Series(pd.cut(np.random.randint(0,60,5), bins=[0,10,30,60]))

0     (0, 10]
1     (0, 10]
2    (10, 30]
3    (30, 60]
4    (10, 30]
dtype: category
Categories (3, interval[int64]): [(0, 10] < (10, 30] < (30, 60]]

In [78]:
# 也可以转为 str 等其他类型
pd.Series(pd.cut(np.random.randint(0,60,5), bins=[0,10,30,60])).astype('str')
pd.Series(pd.cut(np.random.randint(0,60,5), bins=[0,10,30,60])).astype('str').astype('string')
# pd.Series(pd.cut(np.random.randint(0,60,5), bins=[0,10,30,60])).astype('string') # 但不能一步到位地转为string # ValueError: StringArray requires a sequence of strings or pandas.NA
pd.Series(pd.cut(np.random.randint(0,60,5), bins=[0,10,30,60])).astype('O')
#pd.Series(pd.cut(np.random.randint(0,60,5), bins=[0,10,30,60])).astype('O').astype('string') # 也不能直接从 O 转为string # ValueError: StringArray requires a sequence of strings or pandas.NA

0     (0, 10]
1     (0, 10]
2    (30, 60]
3    (30, 60]
4    (30, 60]
dtype: object

In [80]:
df_c=pd.DataFrame()
df_c['str']=pd.Series(pd.cut(np.random.randint(0,60,5), bins=[0,10,30,60])).astype('str')
df_c['string']=pd.Series(pd.cut(np.random.randint(0,60,5), bins=[0,10,30,60])).astype('str').astype('string')
df_c['O']=pd.Series(pd.cut(np.random.randint(0,60,5), bins=[0,10,30,60])).astype('O')
df_c['category']=pd.Series(pd.cut(np.random.randint(0,60,5), bins=[0,10,30,60]))
df_c.memory_usage() # 真是奇怪, str 等三列的内存使用竟然比index还少.

Index       128
str          40
string       40
O            40
category    133
dtype: int64

In [80]:
pd.interval_range??
# 用于创建区间类型的索引

Signature:
pd.interval_range(
    start=None,
    end=None,
    periods=None,
    freq=None,
    name=None,
    closed='right',
)
Source:   
def interval_range(
    start=None, end=None, periods=None, freq=None, name=None, closed="right"
):
    """
    Return a fixed frequency IntervalIndex.

    Parameters
    ----------
    start : numeric or datetime-like, default None
        Left bound for generating intervals.
    end : numeric or datetime-like, default None
        Right bound for generating intervals.
    periods : int, default None
        Number of periods to generate.
    freq : numeric, str, or DateOffset, default None
        The length of each interval. Must be consistent with the type of start
        and end, e.g. 2 for numeric, or '5H' for datetime-like.  Default is 1
        for numeric and 'D' for datetime-like.
    name : str, default None
        Name of the resulting IntervalIndex.
    closed : {'left', 'right', 'both', 'neither'}, default 'right'
        Whether 

In [83]:
pd.IntervalIndex??
#

Init signature:
pd.IntervalIndex(
    data,
    closed=None,
    dtype=None,
    copy: bool = False,
    name=None,
    verify_integrity: bool = True,
)
Docstring:     
Immutable index of intervals that are closed on the same side.

.. versionadded:: 0.20.0

Parameters
----------
data : array-like (1-dimensional)
    Array-like containing Interval objects from which to build the
    IntervalIndex.
closed : {'left', 'right', 'both', 'neither'}, default 'right'
    Whether the intervals are closed on the left-side, right-side, both or
    neither.
dtype : dtype or None, default None
    If None, dtype will be inferred.

    .. versionadded:: 0.23.0
copy : bool, default False
    Copy the input data.
name : object, optional
     Name to be stored in the index.
verify_integrity : bool, default True
    Verify that the IntervalIndex is valid.

Attributes
----------
left
right
closed
mid
length
is_empty
is_non_overlapping_monotonic
is_overlapping
values

Methods
-------
from_arrays
from_tupl

In [81]:
# 用于作为索引的区间类型
pd.interval_range(start=0, end=5) # 默认 freq=1(如果是时间的话,freq='D')
#     freq : numeric, str, or DateOffset, default None
#        The length of each interval. Must be consistent with the type of start
#        and end, e.g. 2 for numeric, or '5H' for datetime-like.  Default is 1
#        for numeric and 'D' for datetime-like.

IntervalIndex([(0, 1], (1, 2], (2, 3], (3, 4], (4, 5]],
              closed='right',
              dtype='interval[int64]')

In [40]:
pd.cut??
#     Bin values into discrete intervals. 将值转换为值所在的区间 -- 升采样

Signature:
pd.cut(
    x,
    bins,
    right: bool = True,
    labels=None,
    retbins: bool = False,
    precision: int = 3,
    include_lowest: bool = False,
    duplicates: str = 'raise',
)
Source:   
def cut(
    x,
    bins,
    right: bool = True,
    labels=None,
    retbins: bool = False,
    precision: int = 3,
    include_lowest: bool = False,
    duplicates: str = "raise",
):
    """
    Bin values into discrete intervals.

    Use `cut` when you need to segment and sort data values into bins. This
    function is also useful for going from a continuous variable to a
    categorical variable. For example, `cut` could convert ages to groups of
    age ranges. Supports binning into an equal number of bins, or a
    pre-specified array of bins.

    Parameters
    ----------
    x : array-like
        The input array to be binned. Must be 1-dimensional.
    bins : int, sequence of scalars, or IntervalIndex
        The criteria to bin by.

        * int : Defines the number of

#### 可指定字符为标签

In [82]:
pd.cut(np.random.randint(0,60,5), [0,10,30,60], right=False, labels=['0-10','10-30','30-60'])

[0-10, 10-30, 30-60, 10-30, 30-60]
Categories (3, object): [0-10 < 10-30 < 30-60]

#### （e） 读取 excel,csv等文件时, 通过参数 dtype 创建

In [84]:
df.columns

Index(['School', 'Class', 'ID', 'Gender', 'Address', 'Height', 'Weight',
       'Math', 'Physics'],
      dtype='object')

In [86]:
# 在读取 csv 等文件时, 可以为dtype参数传入字典, 分别为每一列指定 dtype
coltype={'School':'category', 'Class':'category', 'ID':'int', 'Gender':'category', 'Address':'category', 'Height':'float', 'Weight':'float','Math':'float', 'Physics':'category'}
dfc = pd.read_csv('data/table.csv',dtype=coltype)
dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   School   35 non-null     category
 1   Class    35 non-null     category
 2   ID       35 non-null     int32   
 3   Gender   35 non-null     category
 4   Address  35 non-null     category
 5   Height   35 non-null     float64 
 6   Weight   35 non-null     float64 
 7   Math     35 non-null     float64 
 8   Physics  35 non-null     category
dtypes: category(5), float64(3), int32(1)
memory usage: 1.8 KB


### 2. 分类变量的结构
#### 一个分类变量包括三个部分，元素值（values）、分类类别（categories）、是否有序（order）
#### 从上面可以看出，使用cut函数创建的分类变量默认为有序分类变量
#### 下面介绍如何获取或修改这些属性

#### （a）describe方法
#### 该方法描述了一个分类序列的情况，包括非缺失值个数、元素值类别数（不是分类类别数）、最多次出现的元素及其频数

In [87]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.describe()

count     4
unique    3
top       a
freq      2
dtype: object

In [88]:
s_i = pd.Series(pd.Categorical([1, 3, 6, 8, 1, 3, 8, 1, 3, 6, 6, 8,], categories=[1, 3, 6, 8]))
s_i.describe()
# 众所周知,describe函数针对数值型和非数值型的列会有不同的统计方式,对于分类变量,不论原始的dtype是不是数值型,describe函数都会将分类变量当作非数值类型来进行描述
# 实际上, 数值型变量转为分类变量后, 就丧失了数值计算的可能性 -- 从数值变量变成了分类变量, 而分类变量是不能用于数值计算的.

count     12
unique     4
top        8
freq       3
dtype: int64

In [89]:
s_i = pd.Series([1, 3, 6, 8, 1, 3, 8, 1, 3, 6, 6, 8,], dtype='category')
s_i.describe()

count     12
unique     4
top        8
freq       3
dtype: int64

#### （b）categories和ordered属性
#### .cat.ordered 查看分类类别和是否排序

In [90]:
s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [91]:
s.cat.ordered,s_i.cat.ordered

(False, False)

### 3. 类别的修改

#### （a）利用set_categories修改
#### 修改分类，但本身值不会变化

In [92]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
display(s)
s.cat.set_categories(['new_a','c'])
# 如果单元格的值不在传入的list中,就会被设置为缺失值

0      a
1      b
2      c
3      a
4    NaN
dtype: category
Categories (4, object): [a, b, c, d]

0    NaN
1    NaN
2      c
3    NaN
4    NaN
dtype: category
Categories (2, object): [new_a, c]

In [93]:
s.cat.set_categories(['new_a','b','c'])
# 如果单元格的值不在传入的list中,就会被设置为缺失值--在数据库中,如果违背了外键约束,会报错,例如oracle会报错 ora-02291

0    NaN
1      b
2      c
3    NaN
4    NaN
dtype: category
Categories (3, object): [new_a, b, c]

In [94]:
s_cat=s.cat.set_categories(['new_a','a','b','c'])
s_cat
# 注意这时的 categories 属性已经变了

0      a
1      b
2      c
3      a
4    NaN
dtype: category
Categories (4, object): [new_a, a, b, c]

In [96]:
s_cat.cat.categories,s.cat.categories

(Index(['new_a', 'a', 'b', 'c'], dtype='object'),
 Index(['a', 'b', 'c', 'd'], dtype='object'))

In [97]:
s_cat.cat.codes # 缺失值的code用-1表示.

0    1
1    2
2    3
3    1
4   -1
dtype: int8

In [98]:
# .cat.remove_unused_categories 方法会把没有用到的类别删除,但不是就地修改,需要重新赋值
s_cat.cat.remove_unused_categories()
#s_cat

0      a
1      b
2      c
3      a
4    NaN
dtype: category
Categories (3, object): [a, b, c]

In [99]:
# .cat.remove_categories 方法会把用了的类别删除,用缺失值填充
s_cat.cat.remove_categories('a')
#s_cat

0    NaN
1      b
2      c
3    NaN
4    NaN
dtype: category
Categories (3, object): [new_a, b, c]

#### （b）利用rename_categories修改
#### 需要注意的是该方法会把值和分类同时修改

In [101]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
display(s)
print(s.cat.codes)
print(s.cat.categories)
s_replace=s.cat.rename_categories(['new_%s'%i for i in s.cat.categories])
display(s_replace)
print(s_replace.cat.codes)
print(s_replace.cat.categories)
# 值被修改很好理解 -- 值列表里本来存的就是code,code对应的分类改变了,自然会使得值列表也跟着改变 -- 这就像保存的链接,不改变链接本身字面值,只是改变链接指向的ip,点击链接,得到的自然是新的页面.

0      a
1      b
2      c
3      a
4    NaN
dtype: category
Categories (4, object): [a, b, c, d]

0    0
1    1
2    2
3    0
4   -1
dtype: int8
Index(['a', 'b', 'c', 'd'], dtype='object')


0    new_a
1    new_b
2    new_c
3    new_a
4      NaN
dtype: category
Categories (4, object): [new_a, new_b, new_c, new_d]

0    0
1    1
2    2
3    0
4   -1
dtype: int8
Index(['new_a', 'new_b', 'new_c', 'new_d'], dtype='object')


#### 利用字典修改值

In [103]:
s.cat.rename_categories({'a':'new_a','b':'new_b'})
# 当有大量的值需要修改(合并时),利用字典修改的效率,和使用replace方法的效率,应该是前者高吧?

0    new_a
1    new_b
2        c
3    new_a
4      NaN
dtype: category
Categories (4, object): [new_a, new_b, c, d]

In [106]:
%timeit df_xy.xy_category.replace({'中关村文理学院':'北京大学'}) # 5.28 ms ± 573 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
%timeit df_xy.xy_category.cat.rename_categories({'孝陵卫皇家理工大学':'南京理工大学'}) # 304 µs ± 24.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
# 确实要快--cat的rename_categories方法应该是使用的类似修改了指针的方法, 把类别代码对应的值修改了.
# 这很类似于数据库设计中,使用外键约束来降低数据冗余(以及满足3NF)的做法

5.28 ms ± 573 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
304 µs ± 24.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### （c）利用add_categories添加

In [108]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.cat.add_categories(['e'])

0      a
1      b
2      c
3      a
4    NaN
dtype: category
Categories (5, object): [a, b, c, d, e]

#### （d）利用remove_categories移除

In [109]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.cat.remove_categories(['d'])

0      a
1      b
2      c
3      a
4    NaN
dtype: category
Categories (3, object): [a, b, c]

#### （e）删除元素值未出现的分类类型

In [110]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.cat.remove_unused_categories()

0      a
1      b
2      c
3      a
4    NaN
dtype: category
Categories (3, object): [a, b, c]

## 二、分类变量的排序
#### 前面提到，分类数据类型被分为有序和无序，这非常好理解，例如分数区间的高低是有序变量，考试科目的类别一般看做无序变量

### 1. 序的建立

#### （a）一般来说会将一个序列转为有序变量，可以利用as_ordered方法

In [111]:
s = pd.Series(["a", "d", "c", "a"]).astype('category').cat.as_ordered()
s

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [a < c < d]

In [112]:
s.cat.as_ordered??
# 是要赋予一个序结构, 就涉及到排序了. 用的是哪种排序算法? 能否人为手工指定序结构?
# 如果是sql,就可以定义一个临时视图, 或者用case表达式按需求定义序结构.

Signature: s.cat.as_ordered(*args, **kwargs)
Docstring:
Set the Categorical to be ordered.

Parameters
----------
inplace : bool, default False
   Whether or not to set the ordered attribute in-place or return
   a copy of this categorical with ordered set to True.

Returns
-------
Categorical
    Ordered Categorical.
Source:   
            def f(self, *args, **kwargs):
                return self._delegate_method(name, *args, **kwargs)
File:      d:\programdata\anaconda3\lib\site-packages\pandas\core\accessor.py
Type:      method


#### 退化为无序变量，只需要使用as_unordered

In [113]:
s.cat.as_unordered()

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [a, c, d]

#### （b）利用set_categories方法中的order参数

In [69]:
pd.Series(["a", "d", "c", "a"]).astype('category').cat.set_categories(['a','c','d'],ordered=True)
# ordered 参数增加序结构

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [a < c < d]

In [145]:
pd.Series(["a", "d", "c", "a"]).astype('category').cat.set_categories(['c','d','a'],ordered=True)
# 序结构和给 set_categories 传入的list 中的元素顺序一致.

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [c < d < a]

In [114]:
s.cat.set_categories??
# ordered : bool, default False
#    Whether or not the categorical is treated as a ordered categorical.
#    If not given, do not change the ordered information.

Signature: s.cat.set_categories(*args, **kwargs)
Docstring:
Set the categories to the specified new_categories.

`new_categories` can include new categories (which will result in
unused categories) or remove old categories (which results in values
set to NaN). If `rename==True`, the categories will simple be renamed
(less or more items than in old categories will result in values set to
NaN or in unused categories respectively).

This method can be used to perform more than one action of adding,
removing, and reordering simultaneously and is therefore faster than
performing the individual steps via the more specialised methods.

On the other hand this methods does not do checks (e.g., whether the
old categories are included in the new categories on a reorder), which
can result in surprising changes, for example when using special string
dtypes, which does not considers a S1 string equal to a single char
python string.

Parameters
----------
new_categories : Index-like
   The categories

#### （c）利用reorder_categories方法
#### 这个方法的特点在于，新设置的分类必须与原分类为同一集合

In [150]:
s = pd.Series(["a", "d", "c", "a"]).astype('category')
s

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [a, c, d]

In [151]:
s.cat.reorder_categories(['a','c','d'],ordered=True)
s

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [a, c, d]

In [153]:
#s.cat.reorder_categories(['a','c'],ordered=True) #不能比原始分类变量的种类少 #报错 ValueError: items in new_categories are not the same as in old categories
#s.cat.reorder_categories(['a','c','d','e'],ordered=True) #不能比原始分类变量的种类多 #报错 ValueError: items in new_categories are not the same as in old categories
#s.cat.reorder_categories(['a','d','c'],ordered=True) # 改变传入的list中元素的顺序, 则改变了序结构.
#s.cat.reorder_categories(('a','d','c'),ordered=True) # 传入tuple,也能指定新的序结构
#s.cat.reorder_categories(('a','d','a','c','a'),ordered=True) # 传入tuple--不能有重复元素 #ValueError: Categorical categories must be unique
#s.cat.reorder_categories(set(('a','c','d')),ordered=True) # 传入 set
#s.cat.reorder_categories(set(('a','c','d','a')),ordered=True) # 传入 set--元素如果重复出现, 情况有些微妙: a,c 确定了 a 在 c 前, d,a 确定了 d 在 a 前,结果最终的序结构是 d<a<c.
s.cat.reorder_categories(set(('a','c','a','d','a')),ordered=True) # 传入 set--

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [d < a < c]

In [161]:
setlist=set(('c','a','d','a')),set(('a','c','d','a')),set(('d','c','a','a')),set(('a','d','a','c')),set(('c','d','a'))
setlist

({'a', 'c', 'd'},
 {'a', 'c', 'd'},
 {'a', 'c', 'd'},
 {'a', 'c', 'd'},
 {'a', 'c', 'd'})

In [169]:
for st in setlist:
    print(s.cat.reorder_categories(st,ordered=True).cat.categories)
# 传入集合的话,序结构就很奇特.

Index(['a', 'd', 'c'], dtype='object')
Index(['d', 'a', 'c'], dtype='object')
Index(['a', 'd', 'c'], dtype='object')
Index(['d', 'a', 'c'], dtype='object')
Index(['a', 'd', 'c'], dtype='object')


In [173]:
listlist=[['c','a','d'],['a','c','d'],['d','c','a'],['a','d','c'],['c','d','a']]
listlist

[['c', 'a', 'd'],
 ['a', 'c', 'd'],
 ['d', 'c', 'a'],
 ['a', 'd', 'c'],
 ['c', 'd', 'a']]

In [174]:
for st in listlist:
    print(s.cat.reorder_categories(st,ordered=True).cat.categories)
# 传入list或者tuple就没那么多奇怪的事情.

Index(['c', 'a', 'd'], dtype='object')
Index(['a', 'c', 'd'], dtype='object')
Index(['d', 'c', 'a'], dtype='object')
Index(['a', 'd', 'c'], dtype='object')
Index(['c', 'd', 'a'], dtype='object')


### 2. 排序

#### 先前在第1章介绍的值排序和索引排序都是适用的

In [175]:
s = pd.Series(np.random.choice(['perfect','good','fair','bad','awful'],50)).astype('category')
s.cat.set_categories(['perfect','good','fair','bad','awful'][::-1],ordered=True).head()

0    fair
1    fair
2     bad
3    good
4    fair
dtype: category
Categories (5, object): [awful < bad < fair < good < perfect]

In [125]:
s.cat.categories, ['perfect','good','fair','bad','awful'][::-1], s.cat.categories==['perfect','good','fair','bad','awful'][::-1]

(Index(['awful', 'bad', 'fair', 'good', 'perfect'], dtype='object'),
 ['awful', 'bad', 'fair', 'good', 'perfect'],
 array([ True,  True,  True,  True,  True]))

In [176]:
s.cat.set_categories(['perfect','good','fair','bad','awful'],ordered=True).head()

0    fair
1    fair
2     bad
3    good
4    fair
dtype: category
Categories (5, object): [perfect < good < fair < bad < awful]

In [178]:
s.sort_values(ascending=False).head(10)
# 按值排序,降序排列,会按照分类变量的序结构来进行排序.

49    perfect
41    perfect
29    perfect
33    perfect
34    perfect
23    perfect
10    perfect
36    perfect
47    perfect
48       good
dtype: category
Categories (5, object): [awful, bad, fair, good, perfect]

In [179]:
# 由于有序结构, 就可以进行排序和排名
s.rank(method='dense',ascending=False).head()

0    3.0
1    3.0
2    4.0
3    2.0
4    3.0
dtype: float64

In [180]:
df_sort = pd.DataFrame({'cat':s.values,'value':np.random.randn(50)}).set_index('cat')
df_sort.head()

,value
cat,
fair,-0.895077
fair,0.078977
bad,1.808059
good,1.184074
fair,0.280166


In [181]:
df_sort.sort_index().head()

,value
cat,
awful,-0.432152
awful,-0.233041
awful,-0.152345
awful,0.721618
awful,0.527053


In [182]:
df_sort.rank(method='dense',ascending=False).head() # 排名后的名次是 float 就很讨厌

,value
cat,
fair,44.0
fair,26.0
bad,2.0
good,7.0
fair,23.0


In [184]:
df_sort.rank(method='dense',ascending=False).astype('int').head() # 使用astype 转为int

,value
cat,
fair,44
fair,26
bad,2
good,7
fair,23


## 三、分类变量的比较操作

### 1. 与标量或等长序列的比较

#### （a）标量比较

In [185]:
s = pd.Series(["a", "d", "c", "a"]).astype('category')
s == 'a' # 广播机制

0     True
1    False
2    False
3     True
dtype: bool

#### （b）等长序列比较

In [186]:
s == list('abcd'),s == list('adca'),

(0     True
 1    False
 2     True
 3    False
 dtype: bool,
 0    True
 1    True
 2    True
 3    True
 dtype: bool)

In [187]:
s.equals('a'),s.values.equals(list('abcd')),s.values.equals(list('adca')),s.equals(list('adca')),s.equals(pd.Series(list('adca'))), s.equals(pd.Series(list('adca'),dtype='category')),
# dtype 不一致, 也会导致不相等

(False, False, False, False, False, True)

### 2. 与另一分类变量的比较

#### （a）等式判别（包含等号和不等号）
#### 两个分类变量的等式判别需要满足分类完全相同

In [188]:
s = pd.Series(["a", "d", "c", "a"]).astype('category')
s == s

0    True
1    True
2    True
3    True
dtype: bool

In [189]:
s != s

0    False
1    False
2    False
3    False
dtype: bool

In [192]:
s_new = s.cat.set_categories(['a','d','e'])
#s == s_new #报错 TypeError: Categoricals can only be compared if 'categories' are the same.
#这种情况直接反馈个False不行吗?

#### （b）不等式判别（包含>=,<=,<,>）
#### 两个分类变量的不等式判别需要满足两个条件：① 分类完全相同 ② 排序完全相同

In [197]:
s = pd.Series(["a", "d", "c", "a"]).astype('category')
#s >= s #报错 TypeError: Unordered Categoricals can only compare equality or not

In [198]:
s = pd.Series(["a", "d", "c", "a"]).astype('category').cat.reorder_categories(['a','c','d'],ordered=True)
s >= s, s == s , s > s, s <= s, s < s

(0    True
 1    True
 2    True
 3    True
 dtype: bool,
 0    True
 1    True
 2    True
 3    True
 dtype: bool,
 0    False
 1    False
 2    False
 3    False
 dtype: bool,
 0    True
 1    True
 2    True
 3    True
 dtype: bool,
 0    False
 1    False
 2    False
 3    False
 dtype: bool)

## 四、问题与练习

#### 【问题一】 如何使用union_categoricals方法？它的作用是什么？

In [214]:
a = pd.Categorical(["b", "c"])
b = pd.Categorical(["a", "b"])
pd.concat([pd.Series(a),pd.Series(b)])

0    b
1    c
0    a
1    b
dtype: object

In [216]:
#pd.concat([a,b]) #TypeError: cannot concatenate object of type '<class 'pandas.core.arrays.categorical.Categorical'>'; only Series and DataFrame objs are valid
#必须先把分类数据转换为Series后才能使用pd.concat
pd.concat([pd.Series(a),pd.Series(b)]).astype('category')
#concat后,再转为分类数据,效果与下述函数一致吗?

0    b
1    c
0    a
1    b
dtype: category
Categories (3, object): [a, b, c]

In [217]:
from pandas.api.types import union_categoricals 
union_categoricals([a, b])
# 合并时,会去重

[b, c, a, b]
Categories (3, object): [b, c, a]

In [218]:
type(union_categoricals([a, b]))
# 结果的数据类型

pandas.core.arrays.categorical.Categorical

In [219]:
# 和union之前的对象的类型一致
type(union_categoricals([a, b]))==type(a)

True

In [220]:
# 转换效果对比--效果一致
pd.concat([pd.Series(a),pd.Series(b)]).astype('category')==union_categoricals([a, b])

0    True
1    True
0    True
1    True
dtype: bool

In [221]:
# 转换效率对比--union_categorials 方法的速度要快一倍
%timeit pd.concat([pd.Series(a),pd.Series(b)]).astype('category')
%timeit union_categoricals([a, b])

1.27 ms ± 42.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
755 µs ± 35.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [174]:
union_categoricals??

Signature:
union_categoricals(
    to_union,
    sort_categories: bool = False,
    ignore_order: bool = False,
)
Source:   
def union_categoricals(
    to_union, sort_categories: bool = False, ignore_order: bool = False
):
    """
    Combine list-like of Categorical-like, unioning categories.

    All categories must have the same dtype.

    Parameters
    ----------
    to_union : list-like
        Categorical, CategoricalIndex, or Series with dtype='category'.
    sort_categories : bool, default False
        If true, resulting categories will be lexsorted, otherwise
        they will be ordered as they appear in the data.
    ignore_order : bool, default False
        If true, the ordered attribute of the Categoricals will be ignored.
        Results in an unordered categorical.

    Returns
    -------
    Categorical

    Raises
    ------
    TypeError
        - all inputs do not have the same dtype
        - all inputs do not have the same ordered property
        - all input

#### 【问题二】 利用concat方法将两个序列纵向拼接，它的结果一定是分类变量吗？什么情况下不是？

In [222]:
s1 = pd.Series(['a', 'b'],dtype='category')
s11 = pd.Series(['a', 'b', 'a', 'b'],dtype='category')
s2 = pd.Series(['c', 'd'],dtype='category')
s22 = pd.Series(['a', 'b', 'c', 'd'],dtype='category')

display(s1,s2)
pd.concat([s1, s2])
# concat 两个类别不一致的分类变量后, 分类变量变成object类型了.

0    a
1    b
dtype: category
Categories (2, object): [a, b]

0    c
1    d
dtype: category
Categories (2, object): [c, d]

0    a
1    b
0    c
1    d
dtype: object

In [223]:
display(s1,s11)
pd.concat([s1, s11])
# concat 两个类别完全一致的分类变量后, 分类变量变成object类型了.

0    a
1    b
dtype: category
Categories (2, object): [a, b]

0    a
1    b
2    a
3    b
dtype: category
Categories (2, object): [a, b]

0    a
1    b
0    a
1    b
2    a
3    b
dtype: category
Categories (2, object): [a, b]

In [224]:
display(s2,s22)
pd.concat([s2, s22])
# concat 两个有重合但不一致的分类变量后, 分类变量变成object类型了.

0    c
1    d
dtype: category
Categories (2, object): [c, d]

0    a
1    b
2    c
3    d
dtype: category
Categories (4, object): [a, b, c, d]

0    c
1    d
0    a
1    b
2    c
3    d
dtype: object

In [225]:
display(s2,s22)
pd.concat([s22, s2])
# concat 两个有重合但不一致的分类变量后, 分类变量变成object类型了--改变顺序也没有用的.

0    c
1    d
dtype: category
Categories (2, object): [c, d]

0    a
1    b
2    c
3    d
dtype: category
Categories (4, object): [a, b, c, d]

0    a
1    b
2    c
3    d
0    c
1    d
dtype: object

In [226]:
# 使用 union_categoricals 方法, 直接得到一个将分类按集合的并集运算合并的分类,并且保持合并结果为分类变量--并且速度比曲线救国更快.
union_categoricals([s1, s2]),type(union_categoricals([s1, s2]))

([a, b, c, d]
 Categories (4, object): [a, b, c, d],
 pandas.core.arrays.categorical.Categorical)

In [227]:
union_categoricals([s1, s2])==pd.concat([s1,s2]).astype('category')

0    True
1    True
0    True
1    True
dtype: bool

In [228]:
%timeit union_categoricals([s1, s2])
%timeit pd.concat([s1,s2]).astype('category')

820 µs ± 66.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.16 ms ± 27.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### 【问题三】 当使用groupby方法或者value_counts方法时，分类变量的统计结果和普通变量有什么区别？


In [232]:
# 再举一个长字符串的例子
xy=['孝陵卫皇家理工大学','成都七中附属大学','虹梅南路女子高等师范专科学院','常乐村男子职业技术学校','五道口男子职业技术学院','常乐村男子职业技术学校','翔安皇家理工大学（翔安校区）','中关村文理学院','老和山职业技术学校','沙坡村职业技术学院']
df_xy=pd.DataFrame()
xy_arr=np.random.choice(a=xy,size=100000)# 使用np.random.choice 构造长度10000的Series, 值域为 xy 里的10个值.
# 分别构造各种数据类型的列
df_xy['xy_o']=pd.Series(xy_arr,dtype='O') 
df_xy['xy_str']=pd.Series(xy_arr,dtype='str').astype('str') # 即便加了 astype('str'), 该列的类型还是 object 
df_xy['xy_string']=pd.Series(xy_arr,dtype='string')
df_xy['xy_category']=pd.Series(xy_arr,dtype='category')
# 查看各列的内存使用情况.
df_xy.memory_usage()
# 可以看到, category 类型确实更加节省内存--他的实现方式是对该列的(十个)唯一值进行编码,然后保存一份编码和真实学院的对应关系,再在每个位置保存学院对应的编码.

Index             128
xy_o           800000
xy_str         800000
xy_string      800000
xy_category    100392
dtype: int64

In [233]:
# 使用不同列进行 groupby 操作的耗时
df_xy['cat_codes']= df_xy.xy_category.cat.codes

%timeit df_xy.groupby(df_xy.xy_o)
%timeit df_xy.groupby(df_xy.xy_str)
%timeit df_xy.groupby(df_xy.xy_string)
%timeit df_xy.groupby(df_xy.xy_category) # 为什么用分类数据来分组反而更慢了呢?
%timeit df_xy.groupby(df_xy.xy_category.cat.codes) # 特别是用分类数据的代码分组最慢.
%timeit df_xy.groupby(df_xy.cat_codes) # 先将 cat.codes 作为列放到df_xy中,再按该列分组,就比较快了.
%timeit df_xy.xy_category.cat.codes # 这一步就比直接按xy_o,xy_str,xy_string分组更费时了.

38 µs ± 2.11 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
38.6 µs ± 4.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
39.4 µs ± 2.87 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
117 µs ± 5.04 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
850 µs ± 44.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
38.2 µs ± 2.45 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
72.1 µs ± 2.81 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [235]:
# value_counts 的效率, 分类变量更高.
%timeit df_xy.xy_o.value_counts()
%timeit df_xy.xy_category.value_counts()

10.9 ms ± 786 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.12 ms ± 112 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [236]:
# 使用不同列 groupby 后继续进行后续操作的时间对比
# 以 count 函数为例:

%timeit df_xy.groupby(df_xy.xy_o).count()
%timeit df_xy.groupby(df_xy.xy_str).count()
%timeit df_xy.groupby(df_xy.xy_string).count()
%timeit df_xy.groupby(df_xy.xy_category).count() # 第三
%timeit df_xy.groupby(df_xy.xy_category.cat.codes).count() # 第二
%timeit df_xy.groupby(df_xy.cat_codes).count() # 先将 cat.codes 作为列放到df_xy中 -- 第四
%timeit df_xy.xy_category.cat.codes.count() # 最快--

30.8 ms ± 2.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
31.2 ms ± 1.89 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
36.8 ms ± 3.39 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
23 ms ± 1.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
18.1 ms ± 421 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
27.9 ms ± 2.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
249 µs ± 25.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [270]:
# value_counts 函数的效果对比:

display(df_xy.groupby(df_xy.xy_o).count())
display(df_xy.groupby(df_xy.xy_str).count())
display(df_xy.groupby(df_xy.xy_string).count())
display(df_xy.groupby(df_xy.xy_category).count()) 
display(df_xy.groupby(df_xy.xy_category.cat.codes).count()) 
display(df_xy.groupby(df_xy.cat_codes).count()) 
display(df_xy.xy_category.cat.codes.count()) 
# 结果上并没有什么区别

,xy_str,xy_string,xy_category,cat_codes
xy_o,,,,
中关村文理学院,9966,9966,9966,9966
五道口男子职业技术学院,9981,9981,9981,9981
孝陵卫皇家理工大学,9893,9893,9893,9893
常乐村男子职业技术学校,20017,20017,20017,20017
成都七中附属大学,9976,9976,9976,9976
沙坡村职业技术学院,10135,10135,10135,10135
翔安皇家理工大学（翔安校区）,9966,9966,9966,9966
老和山职业技术学校,9978,9978,9978,9978
虹梅南路女子高等师范专科学院,10088,10088,10088,10088


,xy_o,xy_string,xy_category,cat_codes
xy_str,,,,
中关村文理学院,9966,9966,9966,9966
五道口男子职业技术学院,9981,9981,9981,9981
孝陵卫皇家理工大学,9893,9893,9893,9893
常乐村男子职业技术学校,20017,20017,20017,20017
成都七中附属大学,9976,9976,9976,9976
沙坡村职业技术学院,10135,10135,10135,10135
翔安皇家理工大学（翔安校区）,9966,9966,9966,9966
老和山职业技术学校,9978,9978,9978,9978
虹梅南路女子高等师范专科学院,10088,10088,10088,10088


,xy_o,xy_str,xy_category,cat_codes
xy_string,,,,
中关村文理学院,9966,9966,9966,9966
五道口男子职业技术学院,9981,9981,9981,9981
孝陵卫皇家理工大学,9893,9893,9893,9893
常乐村男子职业技术学校,20017,20017,20017,20017
成都七中附属大学,9976,9976,9976,9976
沙坡村职业技术学院,10135,10135,10135,10135
翔安皇家理工大学（翔安校区）,9966,9966,9966,9966
老和山职业技术学校,9978,9978,9978,9978
虹梅南路女子高等师范专科学院,10088,10088,10088,10088


,xy_o,xy_str,xy_string,cat_codes
xy_category,,,,
中关村文理学院,9966,9966,9966,9966
五道口男子职业技术学院,9981,9981,9981,9981
孝陵卫皇家理工大学,9893,9893,9893,9893
常乐村男子职业技术学校,20017,20017,20017,20017
成都七中附属大学,9976,9976,9976,9976
沙坡村职业技术学院,10135,10135,10135,10135
翔安皇家理工大学（翔安校区）,9966,9966,9966,9966
老和山职业技术学校,9978,9978,9978,9978
虹梅南路女子高等师范专科学院,10088,10088,10088,10088


,xy_o,xy_str,xy_string,xy_category,cat_codes
0,9966,9966,9966,9966,9966
1,9981,9981,9981,9981,9981
2,9893,9893,9893,9893,9893
3,20017,20017,20017,20017,20017
4,9976,9976,9976,9976,9976
5,10135,10135,10135,10135,10135
6,9966,9966,9966,9966,9966
7,9978,9978,9978,9978,9978
8,10088,10088,10088,10088,10088


,xy_o,xy_str,xy_string,xy_category
cat_codes,,,,
0,9966,9966,9966,9966
1,9981,9981,9981,9981
2,9893,9893,9893,9893
3,20017,20017,20017,20017
4,9976,9976,9976,9976
5,10135,10135,10135,10135
6,9966,9966,9966,9966
7,9978,9978,9978,9978
8,10088,10088,10088,10088


100000

In [271]:
# 使用不同列 groupby 后继续进行后续操作的效果
# 以 count 函数为例:

display(df_xy.xy_o.value_counts())
display(df_xy.xy_str.value_counts())
display(df_xy.xy_string.value_counts())
display(df_xy.xy_category.value_counts()) 
display(df_xy.xy_category.cat.codes.value_counts()) 
display(df_xy.cat_codes.value_counts()) 
#display(df_xy.xy_category.cat.codes.value_count()) 
# 执行结果并没有什么区别

常乐村男子职业技术学校       20017
沙坡村职业技术学院         10135
虹梅南路女子高等师范专科学院    10088
五道口男子职业技术学院        9981
老和山职业技术学校          9978
成都七中附属大学           9976
翔安皇家理工大学（翔安校区）     9966
中关村文理学院            9966
孝陵卫皇家理工大学          9893
Name: xy_o, dtype: int64

常乐村男子职业技术学校       20017
沙坡村职业技术学院         10135
虹梅南路女子高等师范专科学院    10088
五道口男子职业技术学院        9981
老和山职业技术学校          9978
成都七中附属大学           9976
翔安皇家理工大学（翔安校区）     9966
中关村文理学院            9966
孝陵卫皇家理工大学          9893
Name: xy_str, dtype: int64

常乐村男子职业技术学校       20017
沙坡村职业技术学院         10135
虹梅南路女子高等师范专科学院    10088
五道口男子职业技术学院        9981
老和山职业技术学校          9978
成都七中附属大学           9976
中关村文理学院            9966
翔安皇家理工大学（翔安校区）     9966
孝陵卫皇家理工大学          9893
Name: xy_string, dtype: Int64

常乐村男子职业技术学校       20017
沙坡村职业技术学院         10135
虹梅南路女子高等师范专科学院    10088
五道口男子职业技术学院        9981
老和山职业技术学校          9978
成都七中附属大学           9976
翔安皇家理工大学（翔安校区）     9966
中关村文理学院            9966
孝陵卫皇家理工大学          9893
Name: xy_category, dtype: int64

3    20017
5    10135
8    10088
1     9981
7     9978
4     9976
6     9966
0     9966
2     9893
dtype: int64

3    20017
5    10135
8    10088
1     9981
7     9978
4     9976
6     9966
0     9966
2     9893
Name: cat_codes, dtype: int64

In [267]:
# value_counts 的效果
df_xy.apply(lambda x:x.value_counts())

,xy_o,xy_str,xy_string,xy_category,cat_codes
0,NaN,NaN,<NA>,NaN,9966.0
1,NaN,NaN,<NA>,NaN,9981.0
2,NaN,NaN,<NA>,NaN,9893.0
3,NaN,NaN,<NA>,NaN,20017.0
4,NaN,NaN,<NA>,NaN,9976.0
5,NaN,NaN,<NA>,NaN,10135.0
6,NaN,NaN,<NA>,NaN,9966.0
7,NaN,NaN,<NA>,NaN,9978.0
8,NaN,NaN,<NA>,NaN,10088.0
中关村文理学院,9966.0,9966.0,9966,9966.0,NaN


In [269]:
#
df_xy.apply(lambda x:x.groupby(x).count())

,xy_o,xy_str,xy_string,xy_category,cat_codes
中关村文理学院,9966.0,9966.0,9966.0,9966.0,NaN
五道口男子职业技术学院,9981.0,9981.0,9981.0,9981.0,NaN
孝陵卫皇家理工大学,9893.0,9893.0,9893.0,9893.0,NaN
常乐村男子职业技术学校,20017.0,20017.0,20017.0,20017.0,NaN
成都七中附属大学,9976.0,9976.0,9976.0,9976.0,NaN
沙坡村职业技术学院,10135.0,10135.0,10135.0,10135.0,NaN
翔安皇家理工大学（翔安校区）,9966.0,9966.0,9966.0,9966.0,NaN
老和山职业技术学校,9978.0,9978.0,9978.0,9978.0,NaN
虹梅南路女子高等师范专科学院,10088.0,10088.0,10088.0,10088.0,NaN
0,NaN,NaN,NaN,NaN,9966.0


#### 【问题四】 下面的代码说明了Series创建分类变量的什么“缺陷”？如何避免？（提示：使用Series中的copy参数）

In [241]:
cat = pd.Categorical([1, 2, 3, 10], categories=[1, 2, 3, 4, 10])
s = pd.Series(cat, name="cat") # 传入cat,则会用视图,也就是一个指针,指向cat
cat

[1, 2, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

In [242]:
s.iloc[0:2] = 10 # 当对视图进行修改时,会反映到原始内存数据上.
cat# 由于指向的是同一块内存数据,因此cat也就被修改了.
# 这类似于数据库操作中的利用视图更新数据--视图本身是虚拟的表,里边没有数据,但在特定条件下, 视图是可以被更新的, 而更新的结果会反映在原始的表中--很多场合下这种方式是有利的.

[10, 10, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

*  注记:  
Python 存储变量的方法跟其他 OOP 语言不同。它与其说是把值赋给变量，不如说是给变量建立了一个到具体值的 reference。           
当在 Python 中 a = something 应该理解为给 something 贴上了一个标签 a。当再赋值给 a 的时候，就好象把 a 这个标签从原来的 something 上拿下来，贴到其他对象上，建立新的 reference。       
原文链接：https://blog.csdn.net/qq_32907349/article/details/52190796

In [3]:
cat = pd.Categorical([1, 2, 3, 10], categories=[1, 2, 3, 4, 10])
s = pd.Series(cat, name="cat",copy=True) # 传入cat,则会用视图,也就是一个指针,指向cat
cat

[1, 2, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

In [4]:
s.iloc[0:2] = 10 # 当对视图进行修改时,会反映到原始内存数据上.
cat# 由于指向的是同一块内存数据,因此cat也就被修改了.
# 这类似于数据库操作中的利用视图更新数据--视图本身是虚拟的表,里边没有数据,但在特定条件下, 视图是可以被更新的, 而更新的结果会反映在原始的表中--很多场合下这种方式是有利的.

[1, 2, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

In [2]:
pd.Series??

Init signature:
pd.Series(
    data=None,
    index=None,
    dtype=None,
    name=None,
    copy=False,
    fastpath=False,
)
Source:        
class Series(base.IndexOpsMixin, generic.NDFrame):
    """
    One-dimensional ndarray with axis labels (including time series).

    Labels need not be unique but must be a hashable type. The object
    supports both integer- and label-based indexing and provides a host of
    methods for performing operations involving the index. Statistical
    methods from ndarray have been overridden to automatically exclude
    missing data (currently represented as NaN).

    Operations between Series (+, -, /, *, **) align values based on their
    associated index values-- they need not be the same length. The result
    index will be the sorted union of the two indexes.

    Parameters
    ----------
    data : array-like, Iterable, dict, or scalar value
        Contains data stored in Series.

        .. versionchanged:: 0.23.0
           If data is a

In [244]:
# 增加copy参数,从cat副本创建s
cat = pd.Categorical([1, 2, 3, 10], categories=[1, 2, 3, 4, 10])
s = pd.Series(cat.copy, name="cat")
display(cat)
s.iloc[0:2] = 10
cat

[1, 2, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

[1, 2, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

In [273]:
#　Python 中, 切片等操作会提供一个视图,而不是一个拷贝.
# int 类型
a=20
b=a
print(a,b)
print(a is b,a==b)
a=20
print(a,b) # b的值并未随着a的值变动,这就是说,b=a的操作,只是把b这个变量名贴到了最开始a代表的20上.当重新给a赋值时,b这个标签还是贴在20上.
print(a is b)
a=30
print(a,b) # b的值并未随着a的值变动,这就是说,b=a的操作,只是把b这个变量名贴到了最开始a代表的20上.当重新给a赋值时,b这个标签还是贴在20上.
print(a is b)


20 20
True True
20 20
True
30 20
False


In [246]:
# str 类型
a='my name is joy.'
b=a
print(a,b)
a='iam ross.'
print(a,b)

my name is joy. my name is joy.
iam ross. my name is joy.


In [247]:
# lsit 类型--修改切片,则a,b都被修改了
a=[1,3,5,8]
b=a
print(a,b)
a[:2]=[2,2]
print(a,b)

[1, 3, 5, 8] [1, 3, 5, 8]
[2, 2, 5, 8] [2, 2, 5, 8]


In [248]:
# Series 类型
a=pd.Series(np.random.randn(5))
b=a
print(a,b)
a[:1]=5
print(a,b) # 修改切片, a,b都被修改了

0   -0.453619
1   -0.095110
2    0.150117
3    0.273699
4   -1.781272
dtype: float64 0   -0.453619
1   -0.095110
2    0.150117
3    0.273699
4   -1.781272
dtype: float64
0    5.000000
1   -0.095110
2    0.150117
3    0.273699
4   -1.781272
dtype: float64 0    5.000000
1   -0.095110
2    0.150117
3    0.273699
4   -1.781272
dtype: float64


In [249]:
# Series 类型
a=pd.DataFrame(np.random.randn(15).reshape((3,5)))
b=a
display(a,b)
a[0]=0
display(a,b) # 修改切片, a,b都被修改了

,0,1,2,3,4
0,-0.093696,1.098712,0.749923,-0.392176,0.459393
1,-1.517748,-1.443906,0.613988,-0.660141,0.338432
2,0.091203,0.227010,0.503136,-1.029868,-2.115464


,0,1,2,3,4
0,-0.093696,1.098712,0.749923,-0.392176,0.459393
1,-1.517748,-1.443906,0.613988,-0.660141,0.338432
2,0.091203,0.227010,0.503136,-1.029868,-2.115464


,0,1,2,3,4
0,0,1.098712,0.749923,-0.392176,0.459393
1,0,-1.443906,0.613988,-0.660141,0.338432
2,0,0.227010,0.503136,-1.029868,-2.115464


,0,1,2,3,4
0,0,1.098712,0.749923,-0.392176,0.459393
1,0,-1.443906,0.613988,-0.660141,0.338432
2,0,0.227010,0.503136,-1.029868,-2.115464


#### 【练习一】 现继续使用第四章中的地震数据集，请解决以下问题：
#### （a）现在将深度分为七个等级：[0,5,10,15,20,30,50,np.inf]，请以深度等级Ⅰ,Ⅱ,Ⅲ,Ⅳ,Ⅴ,Ⅵ,Ⅶ为索引并按照由浅到深的顺序进行排序。
#### （b）在（a）的基础上，将烈度分为4个等级：[0,3,4,5,np.inf]，依次对南部地区的深度和烈度等级建立多级索引排序。

In [34]:
pd.read_csv('data/Earthquake.csv').head()

,日期,时间,维度,经度,方向,距离,深度,烈度
0,2003.05.20,12:17:44 AM,39.04,40.38,west,0.1,10.0,0.0
1,2007.08.01,12:03:08 AM,40.79,30.09,west,0.1,5.2,4.0
2,1978.05.07,12:41:37 AM,38.58,27.61,south_west,0.1,0.0,0.0
3,1997.03.22,12:31:45 AM,39.47,36.44,south_west,0.1,10.0,0.0
4,2000.04.02,12:57:38 AM,40.80,30.24,south_west,0.1,7.0,0.0


In [251]:
df1=pd.read_csv('data/Earthquake.csv')
df1.head()

,日期,时间,维度,经度,方向,距离,深度,烈度
0,2003.05.20,12:17:44 AM,39.04,40.38,west,0.1,10.0,0.0
1,2007.08.01,12:03:08 AM,40.79,30.09,west,0.1,5.2,4.0
2,1978.05.07,12:41:37 AM,38.58,27.61,south_west,0.1,0.0,0.0
3,1997.03.22,12:31:45 AM,39.47,36.44,south_west,0.1,10.0,0.0
4,2000.04.02,12:57:38 AM,40.80,30.24,south_west,0.1,7.0,0.0


In [252]:
#(a)
df1.index=pd.cut(df1.深度,bins=[0,5,10,15,20,30,50,np.inf],labels=['Ⅰ','Ⅱ','Ⅲ','Ⅳ','Ⅴ','Ⅵ','Ⅶ'])
df1.sort_index(ascending=True)

,日期,时间,维度,经度,方向,距离,深度,烈度
深度,,,,,,,,
Ⅰ,1969.08.29,12:46:05 AM,38.00,36.50,south_east,2.3,5.0,0.0
Ⅰ,2007.02.11,12:43:00 AM,38.41,39.13,south_east,2.2,5.0,4.2
Ⅰ,2007.03.08,12:57:56 AM,39.08,40.40,south_east,2.2,5.0,3.9
Ⅰ,2007.04.14,12:30:37 AM,38.31,39.29,south_east,2.2,5.0,4.2
Ⅰ,2011.05.20,12:39:03 AM,39.11,29.10,south_east,2.2,5.0,3.5
...,...,...,...,...,...,...,...,...
NaN,1977.03.09,12:21:25 AM,36.34,29.04,south_west,15.8,0.0,0.0
NaN,1975.03.22,12:25:25 AM,40.38,25.94,north_east,17.0,0.0,3.9
NaN,1991.10.10,12:07:02 AM,40.41,25.87,north,20.0,0.0,0.0


In [253]:
#(b)
df1=pd.read_csv('data/Earthquake.csv')
df1b=df1[df1.方向=='north'].copy()
df1b.深度=pd.cut(df1b.深度,bins=[0,5,10,15,20,30,50,np.inf],labels=['Ⅰ','Ⅱ','Ⅲ','Ⅳ','Ⅴ','Ⅵ','Ⅶ'])
df1b.烈度=pd.cut(df1b.烈度.astype(float),bins=[0,3,4,5,np.inf])
df1b=df1b.set_index(['深度','烈度']).sort_index()
df1b

日期           时间     维度     经度     方向    距离
深度  烈度                                                            
Ⅰ   (3.0, 4.0]  2013.09.26  12:55:36 AM  38.84  43.58  north   0.3
    (3.0, 4.0]  2017.07.29  12:24:11 AM  37.12  28.59  north   0.5
    (3.0, 4.0]  1987.04.25  12:11:00 AM  39.30  27.92  north   0.6
    (3.0, 4.0]  2013.11.08  12:55:34 AM  38.51  36.78  north   0.6
    (3.0, 4.0]  2009.06.14  12:35:54 AM  39.11  29.03  north   0.7
...                    ...          ...    ...    ...    ...   ...
NaN NaN         1986.10.17  12:10:55 AM  36.96  30.33  north   5.7
    NaN         1979.07.23  12:00:42 AM  38.72  26.42  north   6.3
    NaN         1998.06.26  12:24:06 AM  37.10  28.91  north   6.4
    NaN         1991.10.10  12:07:02 AM  40.41  25.87  north  20.0
    NaN         1978.04.02  12:35:03 AM  42.40  26.60  north  48.5

[576 rows x 6 columns]

In [254]:
df1b.index

MultiIndex([('Ⅰ', (3.0, 4.0]),
            ('Ⅰ', (3.0, 4.0]),
            ('Ⅰ', (3.0, 4.0]),
            ('Ⅰ', (3.0, 4.0]),
            ('Ⅰ', (3.0, 4.0]),
            ('Ⅰ', (3.0, 4.0]),
            ('Ⅰ', (3.0, 4.0]),
            ('Ⅰ', (3.0, 4.0]),
            ('Ⅰ', (3.0, 4.0]),
            ('Ⅰ', (3.0, 4.0]),
            ...
            (nan,        nan),
            (nan,        nan),
            (nan,        nan),
            (nan,        nan),
            (nan,        nan),
            (nan,        nan),
            (nan,        nan),
            (nan,        nan),
            (nan,        nan),
            (nan,        nan)],
           names=['深度', '烈度'], length=576)

#### 【练习二】 对于分类变量而言，调用第4章中的变形函数会出现一个BUG（目前的版本下还未修复）：例如对于crosstab函数，按照[官方文档的说法](https://pandas.pydata.org/pandas-docs/version/1.0.0/user_guide/reshaping.html#cross-tabulations)，即使没有出现的变量也会在变形后的汇总结果中出现，但事实上并不是这样，比如下面的例子就缺少了原本应该出现的行'c'和列'f'。基于这一问题，请尝试设计my_crosstab函数，在功能上能够返回正确的结果。

In [255]:
foo = pd.Categorical(['a', 'b'], categories=['a', 'b', 'c'])
bar = pd.Categorical(['d', 'e'], categories=['d', 'e', 'f'])
pd.crosstab(foo, bar)

col_0,d,e
row_0,,
a,1,0
b,0,1


In [256]:
foo

[a, b]
Categories (3, object): [a, b, c]

In [257]:
bar

[d, e]
Categories (3, object): [d, e, f]

In [258]:
foo.categories.values

array(['a', 'b', 'c'], dtype=object)

In [259]:
def my_crosstab(s1,s2):
    return pd.crosstab(s1,s2).reindex_like(pd.crosstab(s1.categories.values,s2.categories.values))#.fillna(0) # 如果缺失值需要用0来填充, 增加一个fillna(0)
my_crosstab(foo,bar)


col_0,d,e,f
row_0,,,
a,1.0,0.0,NaN
b,0.0,1.0,NaN
c,NaN,NaN,NaN


In [260]:
who

a	 arr	 b	 bar	 cat	 coltype	 df	 df1	 df1b	 
df_c	 df_sort	 df_xy	 dfa	 dfc	 foo	 i	 listlist	 my_crosstab	 
np	 pd	 s	 s1	 s11	 s2	 s22	 s_cat	 s_i	 
s_new	 s_replace	 setlist	 st	 temp_df	 type0	 union_categoricals	 xy	 xy_arr	 

